# COVER

In [ ]:
import os
from PIL import Image
import numpy as np
from skimage import measure

def process_and_save_large_square(image_path, output_folder):
    # Load the image
    image = Image.open(image_path)
    image_np = np.array(image)

    # Extract central region for reference color
    center_x, center_y = image_np.shape[1] // 2, image_np.shape[0] // 2
    region_size = 10
    central_region = image_np[center_y-region_size:center_y+region_size, center_x-region_size:center_x+region_size]
    reference_color = central_region.mean(axis=(0, 1))

    # Define color similarity threshold
    threshold = 50
    distances = np.sqrt(np.sum((image_np - reference_color) ** 2, axis=-1))
    mask = distances < threshold

    # Label connected components
    labeled_mask = measure.label(mask)
    regions = measure.regionprops(labeled_mask)

    # Find the region closest to the center
    center_coords = np.array([center_y, center_x])
    closest_region = None
    min_distance = float('inf')

    for region in regions:
        region_center = np.array(region.centroid)
        distance = np.linalg.norm(region_center - center_coords)
        if distance < min_distance:
            min_distance = distance
            closest_region = region

    # Extract the larger square region
    if closest_region:
        min_row, min_col, max_row, max_col = closest_region.bbox

        # Calculate the center and size of the closest square
        square_center_y = (min_row + max_row) // 2
        square_center_x = (min_col + max_col) // 2
        square_side = max(max_row - min_row, max_col - min_col)

        # Calculate the new square with 3x area
        new_side = int(np.sqrt(3) * square_side)
        new_min_row = max(0, square_center_y - new_side // 2)
        new_max_row = min(image_np.shape[0], square_center_y + new_side // 2)
        new_min_col = max(0, square_center_x - new_side // 2)
        new_max_col = min(image_np.shape[1], square_center_x + new_side // 2)

        # Crop the larger square
        large_square = image.crop((new_min_col, new_min_row, new_max_col, new_max_row))

        # Resize to 256x256
        resized_square = large_square.resize((256, 256))

        # Save the resized square
        output_path = os.path.join(output_folder, os.path.basename(image_path))
        resized_square.save(output_path)

def process_folder_and_save_large_squares(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(input_folder, filename)
            process_and_save_large_square(image_path, output_folder)

# Example usage
input_folder = './data/nokia/temp/NO2'
output_folder = './data/nokia/nokia/NO2/cover/'

process_folder_and_save_large_squares(input_folder, output_folder)

# MIDDLE

In [ ]:
import os
from PIL import Image
import numpy as np
from skimage import measure
from matplotlib.patches import Rectangle
from matplotlib import pyplot as plt

# Function to process each image
def process_image(image_path, output_folder):
    image = Image.open(image_path)
    image_np = np.array(image)

    # Extract central region for reference color
    center_x, center_y = image_np.shape[1] // 2, image_np.shape[0] // 2
    region_size = 10
    central_region = image_np[center_y-region_size:center_y+region_size, center_x-region_size:center_x+region_size]
    reference_color = central_region.mean(axis=(0, 1))

    # Define color similarity threshold
    threshold = 50
    distances = np.sqrt(np.sum((image_np - reference_color) ** 2, axis=-1))
    mask = distances < threshold

    # Label connected components
    labeled_mask = measure.label(mask)
    regions = measure.regionprops(labeled_mask)

    # Find the region closest to the center
    center_coords = np.array([center_y, center_x])
    closest_region = None
    min_distance = float('inf')

    for region in regions:
        region_center = np.array(region.centroid)
        distance = np.linalg.norm(region_center - center_coords)
        if distance < min_distance:
            min_distance = distance
            closest_region = region

    # Extract the closest square
    if closest_region:
        min_row, min_col, max_row, max_col = closest_region.bbox
        closest_square = image_np[min_row:max_row, min_col:max_col]
        closest_square_image = Image.fromarray(closest_square)
        resized_image = closest_square_image.resize((256, 256))

        # Save the extracted square
        output_path = os.path.join(output_folder, os.path.basename(image_path))
        resized_image.save(output_path)

# Main function to process all images in a folder
def process_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(input_folder, filename)
            process_image(image_path, output_folder)

# Example usage
input_folder = './data/nokia/nokia/NO2/cover/'
output_folder = './data/nokia/nokia/NO2/middle'

process_folder(input_folder, output_folder)
